In [25]:
import pandas as pd
import numpy as np

import plotly.express as px

from keplergl import KeplerGl
import geopandas as gpd

import os
import glob

%matplotlib inline

### LOAD DATA

In [28]:
path = ("/Users/pastudilloe/Library/CloudStorage/OneDrive-UniversidadSanFranciscodeQuito/2 - USFQ Data Hub/1 - DATA/")

In [29]:
data_COVID = pd.read_csv(path+'COVID/COVID_Jun2022.csv')
linaje_COVID = pd.read_csv(path+'COVID/Linaje_CommonName.csv')

In [30]:
data_COVID

,GISAID ID,Unnamed: 1,Virus name,FECHA DE COLECCIÓN,FECHA DE SUBIDA A GISAID,PROVINCIA,CIUDAD,COD_Canton,TIPO,Genero,Edad,Linaje,Clado,Mutaciones
0,EPI_ISL_417482,NaN,hCoV-19/Ecuador/HEE-01/2020,3/9/20,3/26/20,Pichincha,QUITO,1701,Hospital,Male,57,B.55,L,Spike E1207V
1,EPI_ISL_422563,NaN,hCoV-19/Ecuador/HGSQ-USFQ-018/2020,3/30/20,4/11/20,Pichincha,QUITO,1701,Hospital,Male,27,B.1.1,GR,"(N_R203K,N_G204R,NSP12_P323L,NSP3_L431F,Spike_..."
2,EPI_ISL_422564,NaN,hCoV-19/Ecuador/HGSQ-USFQ-007/2020,3/30/20,4/11/20,Pichincha,QUITO,1701,Hospital,Male,40,B.1.14,O,"(NSP15_S293T,NS3_Q38P,NS3_V163T,NS3_R122E,NS3_..."
3,EPI_ISL_422565,NaN,hCoV-19/Ecuador/HGSQ-USFQ-010/2020,3/30/20,4/11/20,Pichincha,QUITO,1701,Hospital,Male,39,B.1.14,O,"(NSP15_S293T,NS3_Q38P,NS3_V163T,NS3_R122E,NS3_..."
4,EPI_ISL_471267,NaN,hCoV-19/Ecuador/USFQ-004/2020,3/30/20,6/28/20,Pichincha,QUITO,1701,Hospital,Female,73,B.1.67,G,"(NSP12_P323L,Spike_D614G)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1642,EPI_ISL_13102270,NaN,hCoV-19/Ecuador/USFQ-3479/2022,5/28/22,6/3/22,Pichincha,Quito,1701,Ambulatorio,Male,54,BA.2,GRA,"(NSP5_P132H,NSP3_G489S,Spike_L24del,NSP4_T327I..."
1643,EPI_ISL_13102271,NaN,hCoV-19/Ecuador/USFQ-3450/2022,5/29/22,6/3/22,Pichincha,Quito,1701,Ambulatorio,Female,unknown,BA.2,GRA,"(NSP5_P132H,Spike_S371F,NSP3_G489S,NSP13_R392C..."
1644,EPI_ISL_13102272,NaN,hCoV-19/Ecuador/USFQ-3482/2022,5/29/22,6/3/22,Pichincha,Quito,1701,Ambulatorio,Male,5,BA.2,GRA,"(NSP5_P132H,NSP3_G489S,NSP4_T327I,Spike_S373P,..."
1645,EPI_ISL_13102273,NaN,hCoV-19/Ecuador/USFQ-3438/2022,5/30/22,6/3/22,Pichincha,Quito,1701,Ambulatorio,Female,unknown,BA.2.12.1,GRA,"(NSP5_P132H,NSP3_G489S,Spike_L24del,NSP4_T327I..."


In [4]:
city_coord = pd.read_csv(path+ "GENERALES/EC_cities_coordinates.csv")

In [8]:
Pobl_Cantones = pd.read_csv(path+'/GENERALES/EC_canton_poblacion_latlon.csv')
Pobl_Cantones.columns = ['Provincia', 'Canton', 'COD_Canton', 'point_x', 'point_y', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', 'Pop_2020', '2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030']
Pobl_Cantones = Pobl_Cantones[['Provincia', 'Canton', 'COD_Canton','point_x', 'point_y', 'Pop_2020']]

In [9]:
Pobl_Cantones

,Provincia,Canton,COD_Canton,point_x,point_y,Pop_2020
0,AZUAY,CUENCA,101,-79.010407,-2.907662,636996
1,AZUAY,GIRON,102,-79.159570,-3.166273,13037
2,AZUAY,GUALACEO,103,-78.792143,-2.892781,49104
3,AZUAY,NABON,104,-79.076775,-3.352022,17292
4,AZUAY,PAUTE,105,-78.765439,-2.775424,29214
...,...,...,...,...,...,...
220,SANTA ELENA,LIBERTAD,2402,-80.921287,-2.228078,117767
221,SANTA ELENA,SALINAS,2403,-80.979160,-2.208285,94590
222,ZONAS NO DELIMITADAS,LAS GOLONDRINAS,9001,NaN,NaN,7598
223,ZONAS NO DELIMITADAS,MANGA DEL CURA,9003,NaN,NaN,26548


In [10]:
Pobl_Cantones.head(2)

,Provincia,Canton,COD_Canton,point_x,point_y,Pop_2020
0,AZUAY,CUENCA,101,-79.010407,-2.907662,636996
1,AZUAY,GIRON,102,-79.159570,-3.166273,13037


### DATA PROCESSING

In [11]:
data_COVID = data_COVID[['GISAID ID','FECHA DE COLECCIÓN', 'COD_Canton', 'Linaje']]
data_COVID[['Mes', 'Dia', 'Anio']] = data_COVID['FECHA DE COLECCIÓN'].str.split('/', expand=True)

In [12]:
data_COVID_1 = pd.merge(data_COVID, Pobl_Cantones, how='left', on='COD_Canton')
data_COVID_2 = pd.merge(data_COVID_1, linaje_COVID, how='left', on='Linaje')

linaje_COVID

data_COVID_2['Anio'] = data_COVID_2['Anio'].astype(int)
data_COVID_2['Mes'] = data_COVID_2['Mes'].astype(int)


In [15]:
data_COVID_3 = data_COVID_2.groupby(['Provincia', 'Canton','Anio', 'Mes', 'Linaje','Common_Name', 'point_x', 'point_y']).agg({'GISAID ID':'count'}).reset_index()
data_COVID_3 = data_COVID_3.sort_values(['Anio', 'Mes'], ascending=[True, True])
data_COVID_3['COUNT'] = 1
data_COVID_3

,Provincia,Canton,Anio,Mes,Linaje,Common_Name,point_x,point_y,GISAID ID,COUNT
358,PICHINCHA,QUITO,20,3,B.1.1,S/N,-78.480368,-0.176364,1,1
359,PICHINCHA,QUITO,20,3,B.1.14,S/N,-78.480368,-0.176364,2,1
360,PICHINCHA,QUITO,20,3,B.1.67,S/N,-78.480368,-0.176364,1,1
361,PICHINCHA,QUITO,20,3,B.55,S/N,-78.480368,-0.176364,1,1
148,GUAYAS,GUAYAQUIL,20,4,B.1,S/N,-79.910727,-2.113405,1,1
...,...,...,...,...,...,...,...,...,...,...
527,PICHINCHA,QUITO,22,5,BA.4,Omicron,-78.480368,-0.176364,1,1
644,TUNGURAHUA,AMBATO,22,5,BA.1.1,Omicron,-78.620859,-1.244111,3,1
645,TUNGURAHUA,AMBATO,22,5,BA.2,Omicron,-78.620859,-1.244111,5,1
646,TUNGURAHUA,AMBATO,22,5,BA.2.12.1,S/N,-78.620859,-1.244111,3,1


In [16]:
data_COVID_3_ = data_COVID_3[['Provincia', 'Canton', 'Anio', 'Mes', 'Linaje', 'Common_Name', 'GISAID ID']]
data_COVID_3_

,Provincia,Canton,Anio,Mes,Linaje,Common_Name,GISAID ID
358,PICHINCHA,QUITO,20,3,B.1.1,S/N,1
359,PICHINCHA,QUITO,20,3,B.1.14,S/N,2
360,PICHINCHA,QUITO,20,3,B.1.67,S/N,1
361,PICHINCHA,QUITO,20,3,B.55,S/N,1
148,GUAYAS,GUAYAQUIL,20,4,B.1,S/N,1
...,...,...,...,...,...,...,...
527,PICHINCHA,QUITO,22,5,BA.4,Omicron,1
644,TUNGURAHUA,AMBATO,22,5,BA.1.1,Omicron,3
645,TUNGURAHUA,AMBATO,22,5,BA.2,Omicron,5
646,TUNGURAHUA,AMBATO,22,5,BA.2.12.1,S/N,3


In [17]:
data_COVID_4 = data_COVID_3.groupby(['Provincia', 'Canton','Anio', 'Mes']).agg({'COUNT':'sum'}).reset_index()
data_COVID_4 = data_COVID_4.sort_values(['Anio', 'Mes'], ascending=[True, True])
data_COVID_4

,Provincia,Canton,Anio,Mes,COUNT
168,PICHINCHA,QUITO,20,3,4
60,GUAYAS,GUAYAQUIL,20,4,3
68,GUAYAS,PLAYAS,20,4,1
104,LOS RIOS,BABAHOYO,20,4,3
17,CHIMBORAZO,RIOBAMBA,20,5,1
...,...,...,...,...,...
230,TUNGURAHUA,AMBATO,22,4,6
48,ESMERALDAS,ESMERALDAS,22,5,1
130,MANABI,MANTA,22,5,1
190,PICHINCHA,QUITO,22,5,8


In [19]:
data_COVID_3['new_lineage'] = ~data_COVID_3['Linaje'].duplicated()
data_COVID_5 = data_COVID_3[data_COVID_3.new_lineage != False]
data_COVID_5 = data_COVID_5.groupby(['Provincia', 'Canton','Anio', 'Mes',  'point_x', 'point_y']).agg({'COUNT':'sum'}).reset_index()
data_COVID_5 = data_COVID_5.sort_values(['Anio', 'Mes'], ascending=[True, True])

data_COVID_5

,Provincia,Canton,Anio,Mes,point_x,point_y,COUNT
36,PICHINCHA,QUITO,20,3,-78.480368,-0.176364,4
18,GUAYAS,GUAYAQUIL,20,4,-79.910727,-2.113405,2
20,GUAYAS,PLAYAS,20,4,-80.396604,-2.632555,1
13,GALAPAGOS,SAN CRISTOBAL,20,5,-89.608937,-0.904901,1
37,PICHINCHA,QUITO,20,5,-78.480368,-0.176364,1
...,...,...,...,...,...,...,...
6,CAÑAR,AZOGUES,22,3,-78.850632,-2.751282,1
51,PICHINCHA,QUITO,22,3,-78.480368,-0.176364,1
52,PICHINCHA,QUITO,22,4,-78.480368,-0.176364,2
60,TUNGURAHUA,AMBATO,22,4,-78.620859,-1.244111,2


In [20]:
data_COVID_5['Time_N'] = '20'+data_COVID_5['Anio'].astype(str)+ '-' +data_COVID_5['Mes'].astype(str)+ '-01'
data_COVID_5['datetime'] = data_COVID_5['Time_N'].astype(str) + ' 0:00'

data_COVID_5

,Provincia,Canton,Anio,Mes,point_x,point_y,COUNT,Time_N,datetime
36,PICHINCHA,QUITO,20,3,-78.480368,-0.176364,4,2020-3-01,2020-3-01 0:00
18,GUAYAS,GUAYAQUIL,20,4,-79.910727,-2.113405,2,2020-4-01,2020-4-01 0:00
20,GUAYAS,PLAYAS,20,4,-80.396604,-2.632555,1,2020-4-01,2020-4-01 0:00
13,GALAPAGOS,SAN CRISTOBAL,20,5,-89.608937,-0.904901,1,2020-5-01,2020-5-01 0:00
37,PICHINCHA,QUITO,20,5,-78.480368,-0.176364,1,2020-5-01,2020-5-01 0:00
...,...,...,...,...,...,...,...,...,...
6,CAÑAR,AZOGUES,22,3,-78.850632,-2.751282,1,2022-3-01,2022-3-01 0:00
51,PICHINCHA,QUITO,22,3,-78.480368,-0.176364,1,2022-3-01,2022-3-01 0:00
52,PICHINCHA,QUITO,22,4,-78.480368,-0.176364,2,2022-4-01,2022-4-01 0:00
60,TUNGURAHUA,AMBATO,22,4,-78.620859,-1.244111,2,2022-4-01,2022-4-01 0:00


In [22]:
gdf = gpd.GeoDataFrame(data_COVID_5, geometry=gpd.points_from_xy(data_COVID_5.point_x, data_COVID_5.point_y))

In [23]:
#Create a basemap 
map1 = KeplerGl(height=600, width=800)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [24]:
map1.add_data(data=gdf, name='Canton')

#show the map
map1

KeplerGl(data={'Canton': {'index': [36, 18, 20, 13, 37, 38, 26, 39, 61, 40, 23, 41, 24, 54, 21, 27, 28, 42, 59…